In [2]:
import pickle
import dask.dataframe as dd
import pandas as pd

In [ ]:
eICU_features = set(pd.read_csv('../1_importsql/feature_df2.csv').concept_name)
feature_eicu = pd.read_csv('feature_eicu.csv', index_col=0).reset_index(drop=True)
feature_hicu = pd.read_csv('feature_hicu.csv', index_col=0).reset_index(drop=True)

## 매칭 안되는 부분
- drug : 세브란스에서 prescription은 쓸 수 없어서

## 매칭
- fuzzy algorithm 이용해서 비슷한 string에 대해서 매칭 진행하도록 한다.

In [ ]:
feature_eicu.concept_table.unique()

In [ ]:
matching_field = feature_eicu[['concept_name']].merge(feature_hicu[['hicu_feature_english']], how='cross')

In [ ]:
matching_field.head()

In [ ]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def find_matching(eicu_feature, hicu_feature):
    ratio = fuzz.partial_ratio(eicu_feature.lower(), hicu_feature.lower())
    return ratio

In [ ]:
matching_field['match_score'] = matching_field.apply(lambda x : find_matching(x['concept_name'], x['hicu_feature_english']), axis=1)

In [ ]:
matching_field.query('match_score > 80').to_csv('matching.csv', index=False)

일일이 매칭 확인 후 맵핑된 것을 matching.csv로 업로드 하였음.

In [ ]:
matched = pd.read_csv('matching.csv')

In [ ]:
matched

In [ ]:
matched[matched.matching == 1].reset_index(drop=True)

# 이외에 height, weight, vital sign까지 포함해서 common feature로 이용

In [ ]:
hicu_features

In [ ]:
eicu_mapping = pd.read_csv('eicu_mapping.csv')
eicu_mapping_diff = eicu_mapping[eicu_mapping['concept_name'] != eicu_mapping['feature_new_name']]
rename_dict = dict(zip(eicu_mapping_diff.concept_name, eicu_mapping_diff.feature_new_name))
df.rename(rename_dict, axis='columns')

In [ ]:
rename_dict